<div style="
  padding: 30px;
  text-align: center;" class='row'>
<div style="float:left;width: 15%;" class='column'><a href="https://www.colombiacompra.gov.co"><img alt="Logo Colombia Compra Eficiente" id="logocce" src="https://www.colombiacompra.gov.co/sites/cce_public/files/Images/logocce.png" style="height: 45px;"></a></div>
    <div style="float:left;width: 70%;" class='column'>
        <h1>Consulta bases datos abiertos
        </h1> 
    </div>
<div style="float:left;width: 15%;" class='column'><a href="https://www.dnp.gov.co/" target="_blank"><img class="float-right" id="logodnp" src="https://www.colombiacompra.gov.co/sites/cce_public/files/logo_dnp.jpg" style="width: 200px;"></a></div>
</div>

## 1. IDENTIFICACIÓN DEL INSUMO

|||
|:--|:--|
|**Fecha**|Octubre 2022|
|**Ciudad**|Bogotá D.C.|
|**Esquema de presentación del insumo**|Cuaderno Jupyter|
|**Título del insumo**| **Consulta bases datos abiertos**|
|**Descripción y alcance**|Script para consulta bases datos abiertos para paquete.|
|**Periodicidad del insumo**|único|
|**Solicitante**|No aplica|
|**Versión del insumo**|Final|

## 2. DESTINO Y AUTORES DEL INFORME / INSUMO

|||
|:--|:--|
|**Destinatario**|<table align='left'><tr><td>Nombre:</td> <td>Equipo analítica EMAE</td></tr> <tr><td>Cargo:</td> <td>NA</td></tr>  <tr><td>Área:</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE</td></tr></table>|
|**Autores**|<table><tr><td>Nombre:</td> <td>Carlos Isaac Zainea Maya</td></tr> <tr><td>Cargo:</td> <td>Contratista</td></tr>  <tr><td>Área:</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|
|**Aprobación**|<table><tr><td>Nombre:</td> <td>Catalina Pimienta</td></tr> <tr><td>Cargo:</td> <td>Subdirectora Estudios de Mercado y Abastecimiento Estratégico</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|

In [8]:
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json

# Paquetes de consulta a traves de Socrata

from sodapy import Socrata
identificador_columnas=pd.read_csv('datos/identificador_columnas.csv')
valor_columnas={'SECOP I':'API_DA_S_I','SECOP II':'API_DA_S_II','TVEC':'API_DA_S_TVEC'}
identificador_columnas=identificador_columnas[['Unificado','API_DA_S_I', 'API_DA_S_II']]
identificador_columnas=identificador_columnas.dropna()
identificador_columnas=identificador_columnas.set_index('Unificado')
identificador_columnas.drop('Fuente',axis=0,inplace=True)
identificador_datos={'SECOP I':'f789-7hwg','SECOP II':'jbjy-vk9h','TVEC':'rgxm-mmea'}
valor_columnas={'SECOP I':'API_DA_S_I','SECOP II':'API_DA_S_II','TVEC':'API_DA_S_TVEC'}
### Definición de cliente para Socrata

cliente=Socrata('www.datos.gov.co',None,timeout=1000000)

In [9]:
with open('datos/dptos.json') as json_file:
    dptos = json.load(json_file)

In [4]:
def consulta_departamento(departamento):
    results_detalle=[]
    for i in ['SECOP I','SECOP II']:
        queries_list=[]
        results_detalle_dpto=[]
        for j in dptos[departamento]:
            query_secop_tot=""" Select
                {0} as id_contrato,
                {1} as id_proceso,
                {2} as valor_contrato,
                {3} as entidad,
                {4} as departamento_entidad,
                {5} as orden,
                {6} as modalidad,
                {7} as tipo_contrato,
                {8} as codigo_unspsc,
                {9} as objeto_contrato,
                {10} as descripcion_proceso,
                {11} as nombre_proveedor,
                {12} as tipo_documento_proveedor,
                {13} as numero_documento_proveedor,
                {14} as fecha_inicio,
                {15} as fecha_fin,
                {16} as fecha_firma,
                {17} as url

            where
                {4}='{18}'
            limit
                10000000
            """.format(identificador_columnas[valor_columnas[i]]['ID Contrato'],identificador_columnas[valor_columnas[i]]['ID Proceso'],
            identificador_columnas[valor_columnas[i]]['Valor del contrato'],
            identificador_columnas[valor_columnas[i]]['Entidad'],identificador_columnas[valor_columnas[i]]['Departamento Entidad'],
            identificador_columnas[valor_columnas[i]]['Orden'],identificador_columnas[valor_columnas[i]]['Modalidad'],
            identificador_columnas[valor_columnas[i]]['Tipo de contrato'],identificador_columnas[valor_columnas[i]]['UNSPSC'],
            identificador_columnas[valor_columnas[i]]['Objeto Contrato'],identificador_columnas[valor_columnas[i]]['Descripción proceso'],
            identificador_columnas[valor_columnas[i]]['Nombre Proveedor'],identificador_columnas[valor_columnas[i]]['Tipo de documento proveedor'],
            identificador_columnas[valor_columnas[i]]['Documento Proveedor'],identificador_columnas[valor_columnas[i]]['Inicio de contrato'],
            identificador_columnas[valor_columnas[i]]['Fin de contrato'],identificador_columnas[valor_columnas[i]]['Fecha de Firma'],
            identificador_columnas[valor_columnas[i]]['URL'],j)
            queries_list.append(query_secop_tot)
            df_temp=pd.DataFrame(cliente.get(identificador_datos[i], query=query_secop_tot))
            df_temp['fuente']=i
            if df_temp.shape[0]>0:
                results_detalle_dpto.append(df_temp)
        if len(results_detalle_dpto)>0:
            results_detalle.append(pd.concat(results_detalle_dpto))
    results_detalle=pd.concat(results_detalle)
    results_detalle.reset_index(inplace=True)
    results_detalle.drop('index',axis=1,inplace=True)
    results_detalle['valor_contrato']=results_detalle['valor_contrato'].astype(float)
    results_detalle['departamento_entidad']=results_detalle['departamento_entidad'].replace('Distrito Capital de Bogotá','Bogotá DC')\
    .replace('San Andrés Providencia y Santa Catalina','San Andrés, Providencia y Santa Catalina').replace('Norte De Santander','Norte de Santander')
      
    return results_detalle

In [80]:
def consulta_mae(departamento,entidad,fecha_firma_inicio,fecha_firma_fin,valor_min,valor_max,cod_unspsc):
    results_detalle=[]
    
    for i in ['SECOP I','SECOP II']:
        queries_list=[]
        results_detalle_dpto=[]
        if cod_unspsc==None:
            aditional_string=""
        else:
            aditional_string=" and {0} like '%{1}%'".format(identificador_columnas[valor_columnas[i]]['UNSPSC'],cod_unspsc) 
        for j in dptos[departamento]:

            query_secop_tot=""" Select
                {0} as id_contrato,
                {1} as id_proceso,
                {2} as valor_contrato,
                {3} as entidad,
                {4} as departamento_entidad,
                {5} as orden,
                {6} as modalidad,
                {7} as tipo_contrato,
                {8} as codigo_unspsc,
                {9} as objeto_contrato,
                {10} as descripcion_proceso,
                {11} as nombre_proveedor,
                {12} as tipo_documento_proveedor,
                {13} as numero_documento_proveedor,
                {14} as fecha_inicio,
                {15} as fecha_fin,
                {16} as fecha_firma,
                {17} as url

            where
                ({4}='{18}') and
                ({3}='{19}') and
                ({16} between '{20}' and '{21}') and
                ({2} between {22} and {23})  
                {24}
            limit
                10000000
            """.format(identificador_columnas[valor_columnas[i]]['ID Contrato'],identificador_columnas[valor_columnas[i]]['ID Proceso'],
            identificador_columnas[valor_columnas[i]]['Valor del contrato'],
            identificador_columnas[valor_columnas[i]]['Entidad'],identificador_columnas[valor_columnas[i]]['Departamento Entidad'],
            identificador_columnas[valor_columnas[i]]['Orden'],identificador_columnas[valor_columnas[i]]['Modalidad'],
            identificador_columnas[valor_columnas[i]]['Tipo de contrato'],identificador_columnas[valor_columnas[i]]['UNSPSC'],
            identificador_columnas[valor_columnas[i]]['Objeto Contrato'],identificador_columnas[valor_columnas[i]]['Descripción proceso'],
            identificador_columnas[valor_columnas[i]]['Nombre Proveedor'],identificador_columnas[valor_columnas[i]]['Tipo de documento proveedor'],
            identificador_columnas[valor_columnas[i]]['Documento Proveedor'],identificador_columnas[valor_columnas[i]]['Inicio de contrato'],
            identificador_columnas[valor_columnas[i]]['Fin de contrato'],identificador_columnas[valor_columnas[i]]['Fecha de Firma'],
            identificador_columnas[valor_columnas[i]]['URL'],j,entidad,fecha_firma_inicio,fecha_firma_fin,valor_min,valor_max,aditional_string)
            queries_list.append(query_secop_tot)
            df_temp=pd.DataFrame(cliente.get(identificador_datos[i], query=query_secop_tot))
            df_temp['fuente']=i
            print(df_temp.shape)
            if df_temp.shape[0]>0:
                results_detalle_dpto.append(df_temp)
        if len(results_detalle_dpto)>0:
            results_detalle.append(pd.concat(results_detalle_dpto))
    results_detalle=pd.concat(results_detalle)
    results_detalle.reset_index(inplace=True)
    results_detalle.drop('index',axis=1,inplace=True)
    results_detalle['valor_contrato']=results_detalle['valor_contrato'].astype(float)
    results_detalle['departamento_entidad']=results_detalle['departamento_entidad'].replace('Distrito Capital de Bogotá','Bogotá DC')\
    .replace('San Andrés Providencia y Santa Catalina','San Andrés, Providencia y Santa Catalina').replace('Norte De Santander','Norte de Santander')
      
    return results_detalle


In [81]:
df=consulta_mae('Cundinamarca','ALCALDIA ZIPACON','2008-01-01','2022-12-31','0','100000000000','391116')
df

(0, 1)
(1, 19)


,id_contrato,id_proceso,valor_contrato,entidad,departamento_entidad,orden,modalidad,tipo_contrato,codigo_unspsc,objeto_contrato,descripcion_proceso,nombre_proveedor,tipo_documento_proveedor,numero_documento_proveedor,fecha_inicio,fecha_fin,fecha_firma,url,fuente
0,CO1.PCCNTR.907346,CO1.BDOS.751643,82444519.0,ALCALDIA ZIPACON,Cundinamarca,Territorial,Selección Abreviada de Menor Cuantía,Obra,V1.39111600,CONTRATAR LA IMPLEMENTACIÓN DEL PROYECTO DE AL...,CONTRATAR LA IMPLEMENTACIÓN DEL PROYECTO DE AL...,GROUP MLS SAS,NIT,9000681781,2019-04-03T00:00:00.000,2019-07-03T00:00:00.000,2019-04-10T08:04:36.000,{'url': 'https://community.secop.gov.co/Public...,SECOP II


In [5]:
def consulta_datos_departamento(departamento,año):
    df_consulta=pd.read_csv('datos/contratos_entidad.csv')
    df_consulta=df_consulta[df_consulta['departamento_entidad']==departamento]
    df_consulta=df_consulta[df_consulta['año']==año]
    df_consulta['suma_valor_contrato']=df_consulta['suma_valor_contrato'].astype(float)
    df_consulta['cantidad']=df_consulta['cantidad'].astype(int)
    df_consulta['departamento_entidad']=df_consulta['departamento_entidad'].replace('Distrito Capital de Bogotá','Bogotá DC')\
    .replace('San Andrés Providencia y Santa Catalina','San Andrés, Providencia y Santa Catalina').replace('Norte De Santander','Norte de Santander')
    return df_consulta

In [55]:
consulta_departamento('San Andrés, Providencia y Santa Catalina')

,id_contrato,id_proceso,valor_contrato,entidad,departamento_entidad,orden,modalidad,tipo_contrato,codigo_unspsc,objeto_contrato,descripcion_proceso,nombre_proveedor,tipo_documento_proveedor,numero_documento_proveedor,fecha_inicio,fecha_fin,fecha_firma,url,fuente
0,06818,18-4-8778420-7979584,9910000.0,SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA INSTI...,"San Andrés, Providencia y Santa Catalina",TERRITORIAL,Régimen Especial,Obra,721033,"Servicios de Edificación, Construcción de Inst...",SERVICIOS MANTENIMIENTO A TODO COSTO DE INFRAE...,JONATHAN MANUEL PEREZ MERCADO,Cédula de Ciudadanía,1123626969,2018-12-13T00:00:00.000,2018-12-21T00:00:00.000,2018-12-13T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
1,CD7302021,21-12-12139046-11243285,8060000.0,SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA ALCAL...,"San Andrés, Providencia y Santa Catalina",TERRITORIAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,941215,Organizaciones y Clubes,Prestación de servicio de apoyo a la gestión c...,LAUREANA TAYLOR SMITH,Cédula de Ciudadanía,1120980309,2021-02-02T00:00:00.000,2021-07-02T00:00:00.000,2021-02-02T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
2,CD N 020 DE 2016,16-12-4966870-4553694,11737440.0,SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA ALCAL...,"San Andrés, Providencia y Santa Catalina",TERRITORIAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,801116,"Servicios de Gestion, Servicios Profesionales ...",EL CONTRATISTA SE OBLIGA PARA CON EL MUNICIPIO...,FRANCISCO JOSE HUFFINGTON BRITTON,Cédula de Ciudadanía,8724739,2016-02-01T00:00:00.000,2016-08-01T00:00:00.000,2016-02-01T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
3,No definido,16-12-5968329,NaN,SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA ALCAL...,"San Andrés, Providencia y Santa Catalina",TERRITORIAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,801116,"Servicios de Gestion, Servicios Profesionales ...",SERVICIOS TEMPORALES PERSONALES PARA LABORES D...,No Definido,No Definido,No Definido,NaN,NaN,NaN,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
4,541 DE 2015,15-12-3736744-3494849,28922634.0,SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA GOBER...,"San Andrés, Providencia y Santa Catalina",TERRITORIAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,N/D,"Servicios de Gestion, Servicios Profesionales ...",Prestar sus servicios profesionales como Biólo...,CRISTINA ANDREA ZAPATA MASPOLI,Cédula de Ciudadanía,40992274,2015-03-27T00:00:00.000,2015-12-27T00:00:00.000,2015-03-27T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49143,CO1.PCCNTR.1828972,CO1.BDOS.1441050,10055535.0,GOBERNACIÓN DEL DEPARTAMENTO ARCHIPIELAGO DE S...,"San Andrés, Providencia y Santa Catalina",Territorial,Contratación directa,Prestación de servicios,V1.80111600,Prestación de servicios profesionales como adm...,Prestación de servicios profesionales como adm...,DEVI ESTIGUAR ROMERO HERAZO,Cédula de Ciudadanía,1123627320,2020-09-22T00:00:00.000,2020-12-30T00:00:00.000,2020-09-14T16:09:53.000,{'url': 'https://community.secop.gov.co/Public...,SECOP II
49144,CO1.PCCNTR.3542311,CO1.BDOS.2800444,10136565.0,GOBERNACIÓN DEL DEPARTAMENTO ARCHIPIELAGO DE S...,"San Andrés, Providencia y Santa Catalina",Territorial,Contratación directa,Prestación de servicios,V1.80111600,Prestacion de servicios como bachiller academi...,Prestacion de servicios como bachiller academi...,Kelineth Coleth Kelly right,Cédula de Ciudadanía,1007416615,2022-02-02T00:00:00.000,2022-07-01T00:00:00.000,2022-02-01T18:02:30.000,{'url': 'https://community.secop.gov.co/Public...,SECOP II
49145,CO1.PCCNTR.2335241,CO1.BDOS.1833624,34065027.0,GOBERNACIÓN DEL DEPARTAMENTO ARCHIPIELAGO DE S...,"San Andrés, Providencia y Santa Catalina",Territorial,Contratación directa,Prestación de servicios,V1.80111600,PRESTACION DE SERVICIOS PROFESIONALE

## Aprobación
|||
|--|--|
|**Nombre**|Wilson Camilo Sanchez|
|**Cargo**|Gestor T11|
|**Área**|Subdirección EMAE|

![version](https://github.com/ANCP-CCE-Analitica/datos_abiertos/blob/main/images/version.png?raw=1)